For theory see [here](Impl - Nearest Neighbours.ipynb)

$N$ points to be clustered around $K$ means.
Assume initial means $m_{1:K}^{(0)}$ are given.
For every time step $t$:
1. Assign points to means based on squared Euclidean distance:
   $$ S_k^{(t)} = \{x: \| x - m_k^{(t)} \| \le \| x - m_l^{(t)} \| \}, \forall 1 \le l \le K.$$

2. Compute the new means:
   $$ m_k^{(t + 1)} = \frac{1}{ | S_k^{(t)} | } \sum_{x \in S_k^{(t)}} x.$$

Run this until convergence, i.e. until assignments do not change. Say that took $T$ steps.

Initialisation strategies:
1. Forgy: choose random $K$ means from the data.
2. Random Partition: randomly assign each point to a cluster, then compute means.

Complexity:
- Time: $O(N*d*K*T)$, $N$ is the number of points, $d$ is the dimensionality of each point, $K$ is the number of means, and $T$ is the number of iterations until convergence.
- Space: 

In [1]:
import torch
import random
from collections import defaultdict

def assign(inputs, means):
    """
    Assigns points to means
    Args:
        inputs: [N, D]
        means: [K, D] current means
    Returns:
        [N] Array of indexes, each ranging from 0 to K.
        Index k represents cluster of inputs[k]
    """
    # [K, D] - [N, 1, D] = [N, K, D]
    # diff[n][k]: point n - mean k
    diff = means - inputs.unsqueeze(1)

    # [N, K, D].sum(dim=2) = [N, K]
    # dist[n][k]: squared distance from point n to centroid k.
    dist = (diff ** 2).sum(dim=2)

    # [N]
    assignments = dist.argmin(dim=1)
    return assignments


def converged(prev_assignments, assignments):
    """
    Args:
    """
    # Think about floating point non-determinism
    return prev_assignments == assignments


def compute_means(inputs, assignments):
    D = inputs.size(dim=1)
    means = torch.zeros([K, D])
    idx_counts = torch.zeros([K])
    for idx in assignments:
        means[idx] += inputs[idx]
        idx_counts[idx] += 1
    means = means / idx_counts.unsqueeze(dim=1)
    # [K, D]
    return means

K = 5 # number of clusters

# [N, D] Inputs
inputs = torch.randn(50, 2)

# Initialisation: choose random K means from the data
# and compute initial assignments.
means = random.sample(range(len(inputs)), K)
prev_assignments = assign(inputs, means)

while True:
    # [N] Array of indexes, each ranging from 0 to K
    # Index k represents cluster of inputs[k]
    assignments = assign(inputs, means)
    
    # Exit if the convergence criterion is met
    if converged(prev_assignments, assignments):
        break
    prev_assignments = assignments
    
    # [K, D] Array of cluster means
    means = compute_means(inputs, assignments)
    

TypeError: unsupported operand type(s) for -: 'list' and 'Tensor'